# Plot individual results
Do a Tigramite plot for each pc alpha in each variable in each gridpoint.

## Specifications

In [ ]:
import sys
import time
import datetime
from pathlib import Path

import utils.utils as utils
from utils.plotting import plot_links

from utils.setup import Setup

argv           = sys.argv[1:]
# argv           = ['-c', 'cfg_pipeline.yml']
# argv           = ['-c', 'cfg_concat_lon120.yml']
# argv           = ['-c', 'cfg_single_lon120.yml']

setup = Setup(argv)

Path(setup.plots_folder).mkdir(parents=True, exist_ok=True)


### Plot every pc alpha of a single file

In [ ]:
# Load data
len_grid = len(setup.gridpoints)
t_start = time.time()
for i_grid, (lat, lon) in enumerate(setup.gridpoints):
    if setup.analysis == "concat":
        print("Analysis of concatenated files is currently unsupported, skipping.")
        break
    idx_lat = setup.idx_lats[i_grid]
    idx_lon = setup.idx_lons[i_grid]
    
    t_start_gridpoint = time.time()
    print(f"Gridpoint {i_grid+1}/{len_grid}:"
          + f"lat={lat} ({idx_lat}), lon={lon} ({idx_lon})")
    for child in setup.var_children:
        print(f"Variable: {child.name}")
        if child.dimensions == 2:
            child_levels = [[setup.levels[-1],0]]
        elif child.dimensions == 3:
            child_levels = setup.children_idx_levs
        for level in child_levels:
            results_file = utils.generate_results_filename_single(
                    child, level[1], lat, lon, setup.ind_test_name,
                    setup.experiment, setup.output_file_pattern,
                    setup.output_folder)
            if not results_file.is_file():
                print(f"Results file {results_file} not found, skipping.")
                continue
            results = utils.load_results(results_file)

            print(f"Plotting links for {child.name} at level {level[1]+1}")
            # Plotting
            for pc_alpha, alpha_result in results.items():
                plot_file = Path(
                        setup.plots_folder,
                        setup.plot_file_pattern.format(
                            var_name = child.name,
                            level = level[1]+1,
                            lat = int(lat),
                            lon = int(lon),
                            pc_alpha = pc_alpha,
                            ind_test = setup.ind_test_name,
                            experiment = setup.experiment
                        )
                )
                if not setup.overwrite and plot_file.is_file():
#                     print(f"Found file {plot_file}, skipping.")
                    continue # Ignore this result
                if len(alpha_result) > 0:
                    t_before_plot_links = time.time()
                    print(f"pc_alpha = {pc_alpha}")
                    print(f"Plotting to {plot_file}")
                    plot_links(alpha_result["links"],
                               alpha_result["var_names"],
                               save_name = plot_file)
                    time_plot_links = datetime.timedelta(
                            seconds = time.time() - t_before_plot_links
                    )
                    print(f"Plotted. Time: {time_plot_links}")
                else:
                    print("Results are empty, skipping.")

    time_point = datetime.timedelta(
            seconds = time.time() - t_start_gridpoint)
    print(f"All links in gridpoint plotted. Time: {time_point}")
total_time = datetime.timedelta(seconds = time.time() - t_start)
print(f"Execution complete. Total time: {total_time}")
    